In [10]:
# Einlesen der Daten

import os
import numpy as np
from tqdm import tqdm_notebook as tqdm
from PIL import Image

def read_images(path):
    files = os.listdir(path)
    files = [file for file in files if file[-4:] == ".jpg"]
    
    files = files[:2000]
    
    images = []
    for file in tqdm(files):
        try:
            image = Image.open(os.path.join(path, file))

            # https://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.resize
            image = image.resize((224, 224), Image.LANCZOS)

            # https://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.convert
            image = image.convert("RGB")

            image = np.asarray(image)

            images.append(image)
        except OSError:
            pass
    
    return images
    
cats = read_images(os.path.join("data", "PetImages", "Cat"))
dogs = read_images(os.path.join("data", "PetImages", "Dog"))

dogs = np.asarray(dogs)
cats = np.asarray(cats)

# X = np.concatenate([dogs, cats]).astype(np.float32) / 255.
X = np.concatenate([dogs, cats])

y_dogs = np.zeros(len(dogs))
y_cats = np.ones(len(cats))

y = np.concatenate([y_dogs, y_cats])

del dogs, cats, y_dogs, y_cats

<ipython-input-10-80f19bd85744>:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for file in tqdm(files):


## Transfer Learning mit VGG16

In [11]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input

X = preprocess_input(X)

In [12]:
vgg16_model = VGG16(include_top=False, input_shape=(224, 224, 3))
vgg16_model.trainable = False

X_after_vgg = vgg16_model.predict(X, verbose=1)

125/125 [==============================] - 29s 137ms/step


In [13]:
X_after_vgg.shape

(3999, 7, 7, 512)

In [14]:
from sklearn.utils import shuffle

X_after_vgg, y = shuffle(X_after_vgg, y)

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

model = Sequential()

model.add(Flatten(input_shape=(7, 7, 512)))
model.add(Dense(1024, activation="relu"))
model.add(Dense(16, activation="relu"))
model.add(Dense(1, activation="sigmoid"))

model.add

model.compile(optimizer=Adam(lr=0.0001), loss="binary_crossentropy", metrics=["acc"])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
dense (Dense)                (None, 1024)              25691136  
_________________________________________________________________
dense_1 (Dense)              (None, 16)                16400     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 17        
Total params: 25,707,553
Trainable params: 25,707,553
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.fit(X_after_vgg, y, epochs=50, batch_size=64, validation_split=0.2, shuffle=True)

Epoch 1/50
50/50 [==============================] - 2s 26ms/step - loss: 1.2546 - acc: 0.8487 - val_loss: 0.1456 - val_acc: 0.9650
Epoch 2/50
50/50 [==============================] - 1s 18ms/step - loss: 0.0548 - acc: 0.9794 - val_loss: 0.1389 - val_acc: 0.9700
Epoch 3/50
50/50 [==============================] - 1s 18ms/step - loss: 0.0270 - acc: 0.9842 - val_loss: 0.1728 - val_acc: 0.9688
Epoch 4/50
50/50 [==============================] - 1s 25ms/step - loss: 0.0150 - acc: 0.9932 - val_loss: 0.1535 - val_acc: 0.9663
Epoch 5/50
50/50 [==============================] - 1s 18ms/step - loss: 0.0058 - acc: 0.9980 - val_loss: 0.1593 - val_acc: 0.9663
Epoch 6/50
50/50 [==============================] - 1s 18ms/step - loss: 0.0055 - acc: 0.9979 - val_loss: 0.2114 - val_acc: 0.9638
Epoch 7/50
50/50 [==============================] - 1s 18ms/step - loss: 0.0041 - acc: 0.9984 - val_loss: 0.1590 - val_acc: 0.9700
Epoch 8/50
50/50 [==============================] - 1s 19ms/step - loss: 0.0031 - a

In [17]:
model.save("transfer-learning-dave.h5")